In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as T

In [2]:
from barlow_twins import BarlowTwins
from torchvision.models import ResNet, resnet18
from custom_mnist import CustomMnist
from tqdm import tqdm

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
model = torch.load('models/model_x.pth')

In [5]:
backbone = torch.load('models/backbone_x.pth')


In [6]:
train_dataset = CustomMnist('datasets/mnist', train=True, transform=T.ToTensor())
test_dataset = CustomMnist('datasets/mnist', train=False, transform=T.ToTensor())

In [7]:
train_dataset.data = train_dataset.data.unsqueeze(dim=1).repeat(1,3,1,1)
test_dataset.data = test_dataset.data.unsqueeze(dim=1).repeat(1,3,1,1)

In [8]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1024, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1024)

In [9]:
class LinearEval(nn.Module):
    """Some Information about LinearEval"""
    def __init__(self, backbone: nn.Module):
        super(LinearEval, self).__init__()
        self.backbone = backbone
        self.linear = nn.Linear(in_features=512, out_features=10, bias=True)

    def forward(self, x):
        x = self.backbone(x)
        x = torch.flatten(x, start_dim=1)
        x = self.linear(x)

        return x

In [10]:
eval_model = LinearEval(backbone=backbone).to(device)
for param in eval_model.backbone.parameters():
    param.requires_grad = False

In [11]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(eval_model.parameters(), lr=0.06, momentum=0.8)

In [12]:
def test_model(model, testloader):
    wrongly_classified = 0
    for i, data in enumerate(testloader, 0):
        total = len(data[0])
        inputs, labels = data
        inputs,labels = inputs.to(device), labels.to(device)

        with torch.no_grad():
            preds = model(inputs).argmax(dim=1)

        wrong = total - (preds == labels).sum()
        wrongly_classified += wrong

    return wrongly_classified / len(test_dataset)

In [13]:
def train(model, train_loader, test_loader, device):
    # loop over the dataset multiple times
    for epoch in range(30):
        running_loss = 0.0
        for i, data in enumerate(tqdm(train_loader), 0):
            inputs, labels = data
            labels = nn.functional.one_hot(labels).float()
            inputs, labels = inputs.to(device), labels.to(device)


            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = eval_model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print('Epoch: {}, Loss: {}, Test error: {}'.format(epoch, running_loss, test_model(eval_model, test_loader)))

    print('Finished Training')

In [14]:
train(eval_model, train_loader, test_loader, device)

100%|██████████| 59/59 [00:06<00:00,  8.88it/s]


Epoch: 0, Loss: 20.877052783966064, Test error: 0.06529999524354935


100%|██████████| 59/59 [00:05<00:00, 10.15it/s]


Epoch: 1, Loss: 11.4639972448349, Test error: 0.05889999866485596


100%|██████████| 59/59 [00:05<00:00, 10.10it/s]


...

Epoch: 27, Loss: 7.436621397733688, Test error: 0.04749999940395355


100%|██████████| 59/59 [00:05<00:00, 10.18it/s]


Epoch: 28, Loss: 7.421942211687565, Test error: 0.04859999939799309


100%|██████████| 59/59 [00:05<00:00, 10.09it/s]


Epoch: 29, Loss: 7.394971892237663, Test error: 0.050200000405311584
Finished Training
